<a href="https://colab.research.google.com/github/KavyaNair01/Machine_Learning/blob/main/humana_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
# for i in range(50):
#   if i<10:
#     tmp=pd.read_csv(f'/content/drive/MyDrive/humana_train.csv/part-0000{i}-87d80db5-e35b-45d3-98e7-77ca63203dc4-c000.csv')
#   else:
#     tmp=pd.read_csv(f'/content/drive/MyDrive/humana_train.csv/part-000{i}-87d80db5-e35b-45d3-98e7-77ca63203dc4-c000.csv')
#   if i==0:
#     df=tmp
#   else:
#     df=pd.concat([df,tmp],axis=0,ignore_index=True)


# df.head()



,id,preventive_visit_gap_ind,cms_frailty_ind,cms_tot_ma_payment_amt,cms_tot_partd_payment_amt,atlas_recfacpth14,atlas_ffrpth14,atlas_fsrpth14,atlas_grocpth14,atlas_povertyallagespct,...,TEX_elig_cnt,TFP_elig_cnt,TRC_(MRP)_elig_cnt,TRC_(PED)_elig_cnt,HEDIS_comp_cnt,Patient_Experience_comp_cnt,Patient_Safety_comp_cnt,HEDIS_elig_cnt,Patient_Experience_elig_cnt,Patient_Safety_elig_cnt
0,1000073,0,0.0,481.0,0.0,0.0,0.0,1.0,0.0,11.0,...,NaN,NaN,NaN,NaN,1.0,2.0,NaN,2.0,4.0,NaN
1,1000096,0,0.0,1017.0,84.0,0.0,0.0,1.0,0.0,22.0,...,NaN,NaN,1.0,1.0,1.0,NaN,2.0,4.0,NaN,2.0
2,1000113,1,0.0,758.0,0.0,0.0,0.0,0.0,0.0,12.0,...,8.0,NaN,NaN,NaN,NaN,28.0,NaN,NaN,36.0,NaN
3,1000240,1,0.0,377.0,245.0,0.0,0.0,0.0,0.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000369,1,0.0,809.0,72.0,0.0,0.0,0.0,0.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/humana_train.csv/Final_train.csv', low_memory=False)

In [3]:
df.shape

(1527904, 379)

In [13]:
df_demo = pd.read_csv('Demographics.csv')

In [1]:
df = df.merge(df_demo[['id','rucc_category']],how = 'left', on = 'id')

NameError: name 'df' is not defined

In [4]:
df.to_csv('/content/drive/MyDrive/humana_train.csv/Final_train.csv')

In [5]:
df['preventive_visit_gap_ind'].value_counts()

,count
preventive_visit_gap_ind,
0,840844
1,687060


In [10]:
null_cols = df.isnull().sum().reset_index()

In [12]:
list(null_cols.loc[null_cols[0]>(0.5*df.shape[0])]['index'])

['lang_spoken_cd',
 'rucc_category',
 'ABA_comp_cnt',
 'ADH_(ACE)_comp_cnt',
 'ADH_(DIAB)_comp_cnt',
 'ADH_(STATIN)_comp_cnt',
 'ART_comp_cnt',
 'ASV_comp_cnt',
 'BCS_comp_cnt',
 'CBP_comp_cnt',
 'CDC_(EYE)_comp_cnt',
 'CDC_(HbA1c)_comp_cnt',
 'CDC_(NPH)_comp_cnt',
 'COA_(FSA)_comp_cnt',
 'COA_(MDR)_comp_cnt',
 'COA_(PNS)_comp_cnt',
 'COL_comp_cnt',
 'COL_(45-50)_comp_cnt',
 'EED_comp_cnt',
 'EGR_comp_cnt',
 'ESA_comp_cnt',
 'ETA_comp_cnt',
 'FMC_comp_cnt',
 'HBD_comp_cnt',
 'KED_comp_cnt',
 'MDR_comp_cnt',
 'MRP_comp_cnt',
 'OMW_comp_cnt',
 'PCR_comp_cnt',
 'RXC_comp_cnt',
 'SBT_comp_cnt',
 'SFT_comp_cnt',
 'SPC_STATIN_comp_cnt',
 'SUPD_comp_cnt',
 'SWT_comp_cnt',
 'TBC_comp_cnt',
 'TEX_comp_cnt',
 'TFP_comp_cnt',
 'TRC_(MRP)_comp_cnt',
 'TRC_(PED)_comp_cnt',
 'ABA_elig_cnt',
 'ADH_(ACE)_elig_cnt',
 'ADH_(DIAB)_elig_cnt',
 'ADH_(STATIN)_elig_cnt',
 'ART_elig_cnt',
 'ASV_elig_cnt',
 'BCS_elig_cnt',
 'CBP_elig_cnt',
 'CDC_(EYE)_elig_cnt',
 'CDC_(HbA1c)_elig_cnt',
 'CDC_(NPH)_elig_cnt',


In [7]:
df.isnull().sum(axis=1)

,0
0,85
1,72
2,74
3,140
4,81
...,...
1527899,74
1527900,70
1527901,75
1527902,112
